In [15]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import datetime
import pandas as pd
import sys
sys.path.insert(0, '/home/users/sansiddh/projects/covid-modelling/')

In [17]:
from models.seir_movement import SEIR_Movement